In [1]:
!pip install seaborn

You should consider upgrading via the 'c:\users\propcloud\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
import warnings
import certifi
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import warnings
warnings.filterwarnings('ignore')
plt.rcParams["figure.figsize"] = (20,10)
import numpy as np

In [3]:
!pip install bnlp_toolkit

You should consider upgrading via the 'c:\users\propcloud\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [4]:
from collections import defaultdict
from collections import  Counter
from bnlp.corpus import stopwords, punctuations, digits

In [5]:
%%time
large_df = pd.read_csv('final_large_data.csv')
large_df.head(3)

CPU times: total: 14.5 s
Wall time: 14.5 s


,status_text,label
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,bangladesh
1,এবারের উইম্বলডনটা স্মরণীয় করে রাখার মিশনেই যে...,sports
2,জাতীয় সংসদে বিএনপি চেয়ারপারসন ও বিরোধীদলীয় ...,bangladesh


In [6]:
def null(data):
    missing_percent = data.isnull().sum() * 100 / len(data)
    total_missing = data.isnull().sum()
    unique = data.nunique()
    not_null = data.notnull().sum()
    df = pd.DataFrame({'Unique Values': unique,
                        'Value Exist': not_null,
                        'Total_Missing': total_missing,
                         'Missing %': missing_percent})
    print('Data Shape: ',data.shape)
    df = df.round(2)
    return df

In [7]:
null(large_df)

Data Shape:  (408475, 2)


,Unique Values,Value Exist,Total_Missing,Missing %
status_text,406882,408432,43,0.01
label,9,408471,4,0.00


In [8]:
main_df = large_df.copy()

In [9]:
main_df['status_text'].head(5)

0    গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...
1    এবারের উইম্বলডনটা স্মরণীয় করে রাখার মিশনেই যে...
2    জাতীয় সংসদে বিএনপি চেয়ারপারসন ও বিরোধীদলীয় ...
3    সহজ পাসওয়ার্ডের কারণে অনলাইন অ্যাকাউন্ট সহজেই...
4    কম্পিউটার মাউসের উদ্ভাবক ডগলাস অ্যাঙ্গেলবার্ট ...
Name: status_text, dtype: object

In [10]:
main_df = main_df.dropna(inplace=False)
null(main_df)

Data Shape:  (408428, 2)


,Unique Values,Value Exist,Total_Missing,Missing %
status_text,406878,408428,0,0.0
label,9,408428,0,0.0


In [11]:
def filters(corpus):
    res = []
    for i in corpus:
        try:
            if i in stopwords:
              continue

            if i in punctuations + '‘' + '’':
              continue

            if i in digits:
              continue

            res.append(i)
        except Exception as e:
            pass

    return res

In [13]:
main_df['status_text'] = main_df['status_text'].str.split()
main_df.head(5)

,status_text,label
0,"[গাজীপুরের, কালিয়াকৈর, উপজেলার, তেলিরচালা, এল...",bangladesh
1,"[এবারের, উইম্বলডনটা, স্মরণীয়, করে, রাখার, মিশ...",sports
2,"[জাতীয়, সংসদে, বিএনপি, চেয়ারপারসন, ও, বিরোধী...",bangladesh
3,"[সহজ, পাসওয়ার্ডের, কারণে, অনলাইন, অ্যাকাউন্ট,...",technology
4,"[কম্পিউটার, মাউসের, উদ্ভাবক, ডগলাস, অ্যাঙ্গেলব...",technology


In [14]:
%%time
try:
    main_df['status_text'] = main_df['status_text'].apply(filters)
except Exception as e:
    pass
main_df.head(5)

CPU times: total: 5min 31s
Wall time: 5min 31s


,status_text,label
0,"[গাজীপুরের, কালিয়াকৈর, উপজেলার, তেলিরচালা, এল...",bangladesh
1,"[এবারের, উইম্বলডনটা, স্মরণীয়, রাখার, মিশনেই, ...",sports
2,"[জাতীয়, সংসদে, বিএনপি, চেয়ারপারসন, বিরোধীদলী...",bangladesh
3,"[সহজ, পাসওয়ার্ডের, কারণে, অনলাইন, অ্যাকাউন্ট,...",technology
4,"[কম্পিউটার, মাউসের, উদ্ভাবক, ডগলাস, অ্যাঙ্গেলব...",technology


In [15]:
main_df.shape

(408428, 2)

In [16]:
data = main_df.copy()
print(data.shape)
data.head(4)

(408428, 2)


,status_text,label
0,"[গাজীপুরের, কালিয়াকৈর, উপজেলার, তেলিরচালা, এল...",bangladesh
1,"[এবারের, উইম্বলডনটা, স্মরণীয়, রাখার, মিশনেই, ...",sports
2,"[জাতীয়, সংসদে, বিএনপি, চেয়ারপারসন, বিরোধীদলী...",bangladesh
3,"[সহজ, পাসওয়ার্ডের, কারণে, অনলাইন, অ্যাকাউন্ট,...",technology


In [17]:
data = data[['status_text', 'label']]
data.dropna(inplace=True)

### Multinomial NB processing

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
#from ruleparser import RuleParser
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
import re
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV

In [19]:
# a dummy function that just returns its input
def identity(x):
    return x

In [20]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, roc_auc_score, cohen_kappa_score

def Multinomial_Naive_Bayes(trainDoc, trainClass, testDoc, testClass, tfIdf):

    # we use a dummy function as tokenizer and preprocessor,
    # since the texts are already preprocessed and tokenized.
    if tfIdf:
        vec = TfidfVectorizer(preprocessor = identity,
                              tokenizer = identity)
   

    # Pipeline combines the vectorizers with a Naive Bayes classifier
    classifier = Pipeline( [('vec', vec),
                            ('cls', MultinomialNB())] )


    # Train the classifier and build a model using the training documents
    classifier.fit(trainDoc, trainClass)
    
    #saving the model
    import joblib
    joblib.dump(classifier, 'NB_model.pkl')
    
    '''
    #Loading the model:
    NB_spam_model = open('NB_model.pkl','rb')
    classifier = joblib.load(NB_spam_model)
    '''
    # Outputs Predicted class for the test set
    testGuess = classifier.predict(testDoc)
    
    try:
        #inputfile = open("input_nb.txt","r", encoding='utf-8')
        valDOC, valLBL = read_corpus('input_nb.txt', use_sentiment=False)
        predz = classifier.predict(valDOC)
        i=0
        for line in valDOC:
            print(line, predz[i])
            i+=1
        #inputfile.close()
    except:
        pass

    prec, rec, f1, tureSum= precision_recall_fscore_support(testClass, testGuess)
    #classguess = classifier.predict(predict)
    #print("predicted class:", classguess)

    # Simply calculates the accuracy score using the Gold Labels and Predicted Labels
    print("Naive Bayes with TF-idf:")
    accuracy = accuracy_score(testClass, testGuess)

    labels = testClass
    guesses = testGuess
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(labels, guesses, average='micro')
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(labels, guesses, average='micro')
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(labels, guesses, average='micro')
    print('F1 score: %f' % f1)
    print("\n")
    # Showing the Confusion Matrix
    print("\nConfusion Matrix:")
    cm = confusion_matrix(testClass, testGuess, labels=classifier.classes_)
    prec, rec, f1, tureSum= precision_recall_fscore_support(testClass, testGuess)
    print(classifier.classes_)
    print(cm)
#     import seaborn as sns
#     sns.heatmap(cm, annot=True)

In [21]:
# this is the main function but you can name it anyway you want
def main():
    DOC = data['status_text'].to_list()
    LBL = data['label'].to_list()
    
    #print(predict)
    print("length of doc =", len(DOC))

    # Splits the dataset into training (90%) and test set(10%)
    split_point = int(0.90*len(DOC))
    trainDoc = DOC[:split_point]
    trainClass = LBL[:split_point]
    testDoc = DOC[split_point:]
    testClass = LBL[split_point:]

    print("Train data: ", len(trainDoc))
    print("Test data: ", len(testDoc))


    # Calling the classifier (use the tf-idf/count feature/vectorizer)
    
    Multinomial_Naive_Bayes(trainDoc, trainClass, testDoc, testClass, tfIdf=True)

In [22]:
%%time
if __name__ == '__main__':
    main()

length of doc = 408428
Train data:  367585
Test data:  40843
Naive Bayes with TF-idf:
Accuracy: 0.650393
Precision: 0.650393
Recall: 0.650393
F1 score: 0.650393



Confusion Matrix:
['bangladesh' 'economy' 'education' 'entertainment' 'international'
 'life-style' 'opinion' 'sports' 'technology']
[[18008     0     0     0     0     0     0     4     0]
 [ 1412    84     0     0     0     0     0     1     0]
 [  350     0     0     0     0     0     0     1     0]
 [ 2881     0     0  1457     0     0     0    31     0]
 [ 3915     0     0     0   102     0     0     4     0]
 [ 1200     0     0     3     0   100     0     4     0]
 [ 2638     0     0     0     6     0     1    11     0]
 [  484     0     0     0     0     0     0  6594     0]
 [ 1328     0     0     0     0     0     0     6   218]]
CPU times: total: 54.6 s
Wall time: 54.6 s
